# Problem Statment

This is purely a scripting task. We want you to provide us with a script which scrapes the latest `n` lectures from MOOKIT for a course, where `n` is an argument provided by the user himself.  
Now what should it scrape? Scrape the name of the lecture, the week, and possibly the link to the lecture if possible. Output the data to a CSV file.

In [36]:
from bs4 import BeautifulSoup
import re, csv

In [41]:
n = int(input("Number of Latest Lectures: ")) # 10 input for now
url = "https://hello.iitk.ac.in/esc201a21/"
url = input("Enter Page URL of form \"https://hello.iitk.ac.in/your-course-name/\" (remember the slash at the end):")

## HTML of site

URL of `url` is stored in [input.html](./input.html).

In [23]:
with open("./input.html", 'r') as f:
    html = f.read()

## Using bs4 to extract name, week number, duration and video link to mookit

In [4]:
soup = BeautifulSoup(html, "lxml")
soup

<html><body class="vsc-initialized">
<div id="MathJax_Message" style="display: none;"></div>
<app-root _nghost-c0="" ng-version="5.2.9">
<div _ngcontent-c0="" class="bigContainer">
<div _ngcontent-c0="" class="courseTitleTopBox">
<div _ngcontent-c0="" class="overlayDiv"></div>
<div _ngcontent-c0="" class="sideMenuButton">
<div _ngcontent-c0="" class="firstBar"></div>
<div _ngcontent-c0="" class="secondBar"></div>
<div _ngcontent-c0="" class="thirdBar"></div>
</div>
<div _ngcontent-c0="" class="courseIconBox">
<a _ngcontent-c0="" href="https://hello.iitk.ac.in/" target="_blank">
<img _ngcontent-c0="" src="https://hello.iitk.ac.in/sites/default/files/iitklogo.png"/>
</a>
</div>
<div _ngcontent-c0="" class="courseTitleAndDescriptionBox">
<div _ngcontent-c0="" class="courseTitle">
<span _ngcontent-c0="">ESC201A: Introduction to Electronics</span>
</div>
<span _ngcontent-c0="" id="timer" style="float: right; color: black;"></span>
<div _ngcontent-c0="" class="courseDescription">
<span _ngco

In [12]:
allLectureNames = [i.contents[0].strip() for i in soup.select(".lectureInfoBoxText")]
len(allLectureNames)

28

In [16]:
requiredLectures = [(len(allLectureNames)-n+i, allLectureNames[len(allLectureNames)-n+i]) for i in range(n)] # 0-indexing ftw
requiredLectures

[(18, 'L1.12_Superposition'),
 (19, 'L1.13_Power Transfer in DC Circuits'),
 (20, 'L2.0_Passive Components'),
 (21, 'L2.1a_Capacitor and Capacitance'),
 (22, 'L2.1b_Inductor and Inductance'),
 (23, 'L2.2_DC ckt switching - initial and final values'),
 (24, 'L2.1c_General Points on C and L'),
 (25, 'L2.3a_Discharging C thorugh R'),
 (26, 'L2.3b_Charging C with V source through R'),
 (27, 'L2.4_RL Transient Analysis')]

In [20]:
print(soup.select(".lectureInfoBoxText")[18].find_next_sibling("div").contents[2].contents[0], allLectureNames[18]) # Get the duration

12:25 L1.12_Superposition


In [21]:
for i, name in requiredLectures:
    print(i, soup.select(".lectureInfoBoxText")[i].find_next_sibling("div").contents[2].contents[0], name)

18 12:25 L1.12_Superposition
19 07:57 L1.13_Power Transfer in DC Circuits
20 10:52 L2.0_Passive Components
21 23:43 L2.1a_Capacitor and Capacitance
22 13:00 L2.1b_Inductor and Inductance
23 21:59 L2.2_DC ckt switching - initial and final values
24 27:05 L2.1c_General Points on C and L
25 19:50 L2.3a_Discharging C thorugh R
26 20:08 L2.3b_Charging C with V source through R
27 17:07 L2.4_RL Transient Analysis


In [33]:
def duration(i):
    return soup.select(".lectureInfoBoxText")[i].find_next_sibling("div").contents[2].contents[0]

In [24]:
print(url + soup.select(".lectureInfoBoxText")[18].parent.parent.attrs["href"]) # Get the link of lecture in mookit itself

https://hello.iitk.ac.in/esc201a21/#/lecture/100


In [32]:
def link(i):
    return url + soup.select(".lectureInfoBoxText")[i].parent.parent.attrs["href"]

In [27]:
for i, name in requiredLectures:
    print(i, "\t", soup.select(".lectureInfoBoxText")[i].find_next_sibling("div").contents[2].contents[0], "\t", name, "\t", url + soup.select(".lectureInfoBoxText")[i].parent.parent.attrs["href"])

18 	 12:25 	 L1.12_Superposition 	 https://hello.iitk.ac.in/esc201a21/#/lecture/100
19 	 07:57 	 L1.13_Power Transfer in DC Circuits 	 https://hello.iitk.ac.in/esc201a21/#/lecture/101
20 	 10:52 	 L2.0_Passive Components 	 https://hello.iitk.ac.in/esc201a21/#/lecture/126
21 	 23:43 	 L2.1a_Capacitor and Capacitance 	 https://hello.iitk.ac.in/esc201a21/#/lecture/128
22 	 13:00 	 L2.1b_Inductor and Inductance 	 https://hello.iitk.ac.in/esc201a21/#/lecture/129
23 	 21:59 	 L2.2_DC ckt switching - initial and final values 	 https://hello.iitk.ac.in/esc201a21/#/lecture/145
24 	 27:05 	 L2.1c_General Points on C and L 	 https://hello.iitk.ac.in/esc201a21/#/lecture/130
25 	 19:50 	 L2.3a_Discharging C thorugh R 	 https://hello.iitk.ac.in/esc201a21/#/lecture/149
26 	 20:08 	 L2.3b_Charging C with V source through R 	 https://hello.iitk.ac.in/esc201a21/#/lecture/150
27 	 17:07 	 L2.4_RL Transient Analysis 	 https://hello.iitk.ac.in/esc201a21/#/lecture/151


In [28]:
week_Loader = soup.select(".lectureInfoBoxText")[0].parent.parent.parent.parent.parent.parent.parent.parent.contents[1]
week_Loader.contents[3].contents[0].strip()

'Week 0'

In [31]:
def weekNumber(i):
    return soup.select(".lectureInfoBoxText")[i].parent.parent.parent.parent.parent.parent.parent.parent.contents[1].contents[3].contents[0].strip()

In [35]:
for i, name in requiredLectures:
    print(i, "\t", duration(i), "\t", weekNumber(i), "\t", name, "\t", link(i))

18 	 12:25 	 Week 2 	 L1.12_Superposition 	 https://hello.iitk.ac.in/esc201a21/#/lecture/100
19 	 07:57 	 Week 2 	 L1.13_Power Transfer in DC Circuits 	 https://hello.iitk.ac.in/esc201a21/#/lecture/101
20 	 10:52 	 Week 3 	 L2.0_Passive Components 	 https://hello.iitk.ac.in/esc201a21/#/lecture/126
21 	 23:43 	 Week 3 	 L2.1a_Capacitor and Capacitance 	 https://hello.iitk.ac.in/esc201a21/#/lecture/128
22 	 13:00 	 Week 3 	 L2.1b_Inductor and Inductance 	 https://hello.iitk.ac.in/esc201a21/#/lecture/129
23 	 21:59 	 Week 3 	 L2.2_DC ckt switching - initial and final values 	 https://hello.iitk.ac.in/esc201a21/#/lecture/145
24 	 27:05 	 Week 3 	 L2.1c_General Points on C and L 	 https://hello.iitk.ac.in/esc201a21/#/lecture/130
25 	 19:50 	 Week 3 	 L2.3a_Discharging C thorugh R 	 https://hello.iitk.ac.in/esc201a21/#/lecture/149
26 	 20:08 	 Week 3 	 L2.3b_Charging C with V source through R 	 https://hello.iitk.ac.in/esc201a21/#/lecture/150
27 	 17:07 	 Week 3 	 L2.4_RL Transient Analysis 

In [38]:
with open("scrapeLectureDetails.csv", "w") as f:
    writer = csv.writer(f)
    for i, name in requiredLectures:
        writer.writerow([i, duration(i), weekNumber(i), name, link(i)])